In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

In [2]:
df = pd.read_csv("output.csv")

In [3]:
predictors = ["surface_code", "pl1_hand_code", "pl2_hand_code", "round_code", "court_code", "series_code", "best_of", "pl1_rank", "pl2_rank", "pl1_pts", "pl2_pts", "pl1_year_pro", "pl2_year_pro", "pl1_weight", "pl2_weight", "pl1_height", "pl2_height", 'pl1_avg_bookmaker_odds', 'pl2_avg_bookmaker_odds', 'pl1_total_games', 'pl1_win_rate', 'pl2_total_games', 'pl2_win_rate', 'pl1_swrate', 'pl2_swrate', 'rank_diff', 'pts_diff', 'wrate_diff', 'swrate_diff']

train = df[df["date"] < "2019-01-01"]
test = df[df["date"] >= "2019-01-01"]

X_train, y_train = train[predictors], train['result']
X_test, y_test = test[predictors], test['result']

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
# definice architektury modelu
model = Sequential([
    Dense(100, activation='sigmoid', input_shape=(X_train_scaled.shape[1],)),
    Dense(50, activation='sigmoid'),
    Dense(25, activation='relu'),
    Dense(10, activation='sigmoid'),
    Dense(1, activation='sigmoid')
])

# kompilace modelu
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# trénink modelu
history = model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2)

# evaluace modelu
y_pred = (model.predict(X_test_scaled) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Epoch 1/10
661/661 [==============================] - 4s 4ms/step - loss: 0.5917 - accuracy: 0.6830 - val_loss: 0.5571 - val_accuracy: 0.7112
Epoch 2/10
661/661 [==============================] - 2s 3ms/step - loss: 0.5670 - accuracy: 0.7069 - val_loss: 0.5547 - val_accuracy: 0.7167
Epoch 3/10
661/661 [==============================] - 2s 3ms/step - loss: 0.5659 - accuracy: 0.7042 - val_loss: 0.5506 - val_accuracy: 0.7148
Epoch 4/10
661/661 [==============================] - 2s 3ms/step - loss: 0.5633 - accuracy: 0.7059 - val_loss: 0.5522 - val_accuracy: 0.7133
Epoch 5/10
661/661 [==============================] - 2s 3ms/step - loss: 0.5632 - accuracy: 0.7055 - val_loss: 0.5508 - val_accuracy: 0.7146
Epoch 6/10
661/661 [==============================] - 2s 3ms/step - loss: 0.5626 - accuracy: 0.7086 - val_loss: 0.5534 - val_accuracy: 0.7120
Epoch 7/10
661/661 [==============================] - 2s 3ms/step - loss: 0.5621 - accuracy: 0.7080 - val_loss: 0.5529 - val_accuracy: 0.7126
Epoch 

In [36]:
# Predict probabilities using the neural network model
pred_probs_nn = model.predict(X_test_scaled)
test_copy = test
# Add the predicted probabilities as a new column to your DataFrame
test['pl1_prob'] = pred_probs_nn
test['pred_result'] = np.where(test['pl1_prob'] >= 0.5, 1, 0)
test['bookmaker_result'] = np.where(test['pl1_avg_bookmaker_odds'] < test['pl2_avg_bookmaker_odds'], 1, 0)

nn_output = test[['id','pl1','pl2','date','pl1_avg_bookmaker_odds','pl2_avg_bookmaker_odds','result','pl1_prob','pred_result','bookmaker_result']]

# Display the updated DataFrame
nn_output.to_excel('nn_output.xlsx')

197/197 [==============================] - 0s 1ms/step


C:\Users\pariz\AppData\Local\Temp\ipykernel_12832\631435607.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pl1_prob'] = pred_probs_nn
C:\Users\pariz\AppData\Local\Temp\ipykernel_12832\631435607.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred_result'] = np.where(test['pl1_prob'] >= 0.5, 1, 0)
C:\Users\pariz\AppData\Local\Temp\ipykernel_12832\631435607.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde